# LAB 1.4: BUILDING PROMPT TEMPLATE LIBRARIES

**Course:** Advanced Prompt Engineering Training  
**Session:** Session 1 - Prompt Engineering Fundamentals Review  
**Duration:** 50 minutes  
**Type:** Hands-on Template Design & Library Development

## LAB OVERVIEW

This lab focuses on building **reusable, maintainable prompt template libraries** for enterprise deployment. You'll learn to:

- Design modular, reusable prompt templates
- Implement variable substitution with validation
- Create composable templates from smaller components
- Version and document templates for team collaboration
- Build production-ready template libraries

**Scenario:** You're the prompt engineering lead at a financial institution. Your team is building 20+ AI features across fraud detection, customer service, loan processing, and compliance. Each feature needs prompts, but they're currently copy-pasting and modifying ad-hoc prompts. You need to build a standardized template library that ensures quality, consistency, and maintainability.

## LEARNING OBJECTIVES

By the end of this lab, you will be able to:

✓ Design reusable prompt templates with clear variable placeholders  
✓ Implement input validation for template variables  
✓ Compose complex templates from modular components  
✓ Version and document templates for enterprise use  
✓ Build searchable template libraries  
✓ Create testing frameworks for template validation

### Step 1: Import Libraries

In [ ]:
# Lab 1.4: Building Prompt Template Libraries
# Advanced Prompt Engineering Training - Session 1

import os
import json
import yaml
from openai import OpenAI
import pandas as pd
from typing import Dict, List, Any, Optional
from string import Template
from jinja2 import Environment, BaseLoader, TemplateError
from datetime import datetime
import re

print("✓ Libraries imported")

### Step 2: Configure OpenAI Client

In [ ]:
# Initialize OpenAI client
api_key=os.environ.get("OPENAI_API_KEY")

# Configuration
MODEL = os.getenv("MODEL_NAME")
TEMPERATURE = 0  # Deterministic for consistent debugging

if not api_key:
    raise ValueError("OPENAI_API_KEY not found. Please set it in .env file")

if not MODEL:
    raise ValueError("MODEL_NAME not found. Please set it in .env file")

client = OpenAI(api_key=api_key)

print(f"✓ Model: {MODEL}")
print(f"✓ Temperature: {TEMPERATURE}")

### Step 3: Create Helper Functions

In [ ]:
def call_gpt4(prompt, system_prompt="You are a helpful AI assistant.", temperature=0):
    """
    Wrapper for GPT-4 API calls
    
    Args:
        prompt (str): User prompt
        system_prompt (str): System prompt
        temperature (float): Sampling temperature
    
    Returns:
        dict: Response with content and metadata
    """
    try:
        response = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
            temperature=temperature
        )
        
        return {
            "content": response.choices[0].message.content,
            "total_tokens": response.usage.total_tokens
        }
    except Exception as e:
        return {
            "content": f"Error: {str(e)}",
            "total_tokens": 0
        }

print("✓ Helper functions created")

### Step 4: Test Connection

In [ ]:
# Test connection
test = call_gpt4("Say 'Ready for template building' if you receive this.")
print(f"Response: {test['content']}")
print("\n✓ Connection verified")

## TEMPLATE DESIGN PRINCIPLES

### Why Templates Matter

**Without templates:**
```python
# Developer 1's fraud detection prompt
prompt1 = "Check if this transaction is fraud: " + transaction

# Developer 2's fraud detection prompt
prompt2 = f"Analyze transaction for fraud indicators:\n{transaction}\nIs it fraudulent?"

# Developer 3's fraud detection prompt
prompt3 = "Transaction: " + transaction + "\n\nFraud analysis needed"
```

**Problems:**
- ❌ Inconsistent quality
- ❌ No reusability
- ❌ Hard to maintain
- ❌ No version control
- ❌ Tribal knowledge



## CHALLENGE 1: BASIC TEMPLATE PATTERNS

**Time:** 10 minutes  
**Objective:** Create basic reusable templates with variable substitution

### Background

Your team needs templates for common BFSI tasks. Start by creating simple templates using Python's string formatting.

### Current Approach (No Templates)

In [ ]:
# BEFORE: Hard-coded, non-reusable prompts

# Fraud detection - Developer 1
def detect_fraud_v1(transaction):
    return f"Is this fraud? {transaction}"

# Fraud detection - Developer 2
def detect_fraud_v2(amount, merchant, location):
    return f"Transaction: ${amount} at {merchant} in {location}. Fraud?"

# Fraud detection - Developer 3
def detect_fraud_v3(data):
    return "Analyze: " + str(data) + " for fraud"

# Problems: Inconsistent, hard to maintain, no quality control

### Student Exercise

In [ ]:
# TODO: Create a PromptTemplate class
# Requirements:
# - Store template string with variables
# - Render template with provided values
# - Validate required variables are provided

class PromptTemplate:
    """
    Basic prompt template with variable substitution
    """
    
    def __init__(self, template: str, name: str = "unnamed"):
        """
        Initialize template
        
        Args:
            template (str): Template string with {variable} placeholders
            name (str): Template name for identification
        """
        # TODO: Implement
        pass
    
    def render(self, **kwargs) -> str:
        """
        Render template with provided variables
        
        Args:
            **kwargs: Variable values
        
        Returns:
            str: Rendered template
        """
        # TODO: Implement
        pass
    
    def get_variables(self) -> List[str]:
        """
        Extract variable names from template
        
        Returns:
            List[str]: Variable names
        """
        # TODO: Implement
        pass

# TODO: Test your PromptTemplate class

### Solution

In [ ]:
# SOLUTION: Basic PromptTemplate Class

class PromptTemplate:
    """
    Basic prompt template with variable substitution
    """
    
    def __init__(self, template: str, name: str = "unnamed"):
        """
        Initialize template
        
        Args:
            template (str): Template string with {variable} placeholders
            name (str): Template name for identification
        """
        self.template = template
        self.name = name
        self.variables = self._extract_variables()
    
    def _extract_variables(self) -> List[str]:
        """Extract variable names from template"""
        # Find all {variable} patterns
        pattern = r'\{(\w+)\}'
        return list(set(re.findall(pattern, self.template)))
    
    def render(self, **kwargs) -> str:
        """
        Render template with provided variables
        
        Args:
            **kwargs: Variable values
        
        Returns:
            str: Rendered template
        
        Raises:
            ValueError: If required variables are missing
        """
        # Check for missing variables
        missing = set(self.variables) - set(kwargs.keys())
        if missing:
            raise ValueError(f"Missing required variables: {missing}")
        
        # Check for extra variables
        extra = set(kwargs.keys()) - set(self.variables)
        if extra:
            print(f"Warning: Extra variables provided (will be ignored): {extra}")
        
        # Render template
        try:
            return self.template.format(**kwargs)
        except KeyError as e:
            raise ValueError(f"Variable substitution failed: {e}")
    
    def get_variables(self) -> List[str]:
        """Get list of required variables"""
        return self.variables
    
    def __str__(self):
        return f"PromptTemplate(name='{self.name}', variables={self.variables})"

# Test the template class
print("BASIC TEMPLATE CLASS:")
print("=" * 80)

# Create fraud detection template
fraud_template = PromptTemplate(
    name="fraud_detection_basic",
    template="""
Analyze this transaction for fraud indicators:

Transaction Amount: ${amount}
Merchant: {merchant}
Location: {location}
Card Location: {card_location}

Assess the fraud risk level: LOW, MEDIUM, or HIGH.
"""
)

print(f"Template: {fraud_template}")
print(f"Required variables: {fraud_template.get_variables()}")
print("\n" + "-" * 80 + "\n")

# Test rendering
test_data = {
    "amount": 1500,
    "merchant": "Electronics Store",
    "location": "Lagos, Nigeria",
    "card_location": "New York, USA"
}

rendered = fraud_template.render(**test_data)
print("Rendered template:")
print(rendered)
print("\n" + "=" * 80)

### Create Template Library

In [ ]:
# Build a basic template library

template_library = {}

# Fraud Detection Template
template_library['fraud_detection'] = PromptTemplate(
    name="fraud_detection",
    template="""
Analyze this transaction for fraud:

Amount: ${amount}
Merchant: {merchant}
Location: {location}
Time: {time}

Determine fraud risk: HIGH, MEDIUM, or LOW.
"""
)

# Credit Risk Assessment Template
template_library['credit_risk'] = PromptTemplate(
    name="credit_risk",
    template="""
Assess credit risk for this applicant:

Credit Score: {credit_score}
Annual Income: ${annual_income}
Existing Debt: ${existing_debt}
Employment Duration: {employment_years} years

Classify as: HIGH_RISK, MODERATE_RISK, or LOW_RISK.
"""
)

# Customer Intent Classification Template
template_library['intent_classification'] = PromptTemplate(
    name="intent_classification",
    template="""
Classify this customer inquiry:

Customer Message: "{message}"

Categories: CARD_SERVICES, LENDING, FRAUD, ACCOUNT_INQUIRY, GENERAL

Output the category only.
"""
)

# Document Summarization Template
template_library['document_summary'] = PromptTemplate(
    name="document_summary",
    template="""
Summarize this {document_type} in {max_words} words or less:

{document_text}

Focus on: {focus_areas}

Provide a concise summary.
"""
)

print("TEMPLATE LIBRARY:")
print("=" * 80)
for name, template in template_library.items():
    print(f"\n{name}:")
    print(f"  Variables: {template.get_variables()}")
print("\n" + "=" * 80)

### Key Takeaways

✓ **Standardization** - One template for each use case  
✓ **Reusability** - Same template across team  
✓ **Maintainability** - Update once, applies everywhere  
✓ **Variable extraction** - Automatic detection of placeholders

## CHALLENGE 2: VARIABLE SUBSTITUTION & VALIDATION

**Time:** 10 minutes  
**Objective:** Add input validation and type checking to templates

### Background

Basic templates work, but they don't prevent invalid inputs. A credit score template shouldn't accept negative numbers or strings like "excellent".

### Problem Demonstration

In [ ]:
# PROBLEM: No validation

credit_template = PromptTemplate(
    name="credit_check",
    template="Credit score: {score}, Risk: {risk_level}"
)

# These should fail but don't:
bad_renders = [
    credit_template.render(score=-500, risk_level="banana"),  # Invalid values
    credit_template.render(score="excellent", risk_level=123),  # Wrong types
    credit_template.render(score=1000, risk_level="HIGH")  # Out of range
]

print("PROBLEM: Templates accept invalid data")
for render in bad_renders:
    print(f"  ✗ {render}")

### Student Exercise

In [ ]:
# TODO: Create ValidatedPromptTemplate with input validation
# Requirements:
# - Define variable types and constraints
# - Validate before rendering
# - Provide helpful error messages

class ValidatedPromptTemplate(PromptTemplate):
    """
    Template with input validation
    """
    
    def __init__(self, template: str, name: str, validators: Dict[str, Dict] = None):
        """
        Initialize template with validators
        
        Args:
            template (str): Template string
            name (str): Template name
            validators (Dict): Validation rules per variable
                Example: {"credit_score": {"type": int, "min": 300, "max": 850}}
        """
        # TODO: Implement
        pass
    
    def _validate_variable(self, var_name: str, value: Any) -> None:
        """
        Validate a single variable
        
        Raises:
            ValueError: If validation fails
        """
        # TODO: Implement
        pass

# TODO: Test validation

### Solution

In [ ]:
# SOLUTION: ValidatedPromptTemplate

class ValidatedPromptTemplate(PromptTemplate):
    """
    Prompt template with input validation
    """
    
    def __init__(
        self, 
        template: str, 
        name: str = "unnamed",
        validators: Dict[str, Dict] = None,
        description: str = ""
    ):
        """
        Initialize validated template
        
        Args:
            template (str): Template string
            name (str): Template name
            validators (Dict): Validation rules
            description (str): Template purpose
        """
        super().__init__(template, name)
        self.validators = validators or {}
        self.description = description
    
    def _validate_variable(self, var_name: str, value: Any) -> None:
        """
        Validate a single variable against its rules
        
        Args:
            var_name (str): Variable name
            value (Any): Variable value
        
        Raises:
            ValueError: If validation fails
        """
        if var_name not in self.validators:
            return  # No validation rules for this variable
        
        rules = self.validators[var_name]
        
        # Type validation
        if "type" in rules:
            expected_type = rules["type"]
            if not isinstance(value, expected_type):
                raise ValueError(
                    f"Variable '{var_name}' must be {expected_type.__name__}, "
                    f"got {type(value).__name__}"
                )
        
        # Numeric range validation
        if "min" in rules and value < rules["min"]:
            raise ValueError(
                f"Variable '{var_name}' must be >= {rules['min']}, got {value}"
            )
        
        if "max" in rules and value > rules["max"]:
            raise ValueError(
                f"Variable '{var_name}' must be <= {rules['max']}, got {value}"
            )
        
        # String length validation
        if "min_length" in rules and len(str(value)) < rules["min_length"]:
            raise ValueError(
                f"Variable '{var_name}' must be at least {rules['min_length']} characters"
            )
        
        if "max_length" in rules and len(str(value)) > rules["max_length"]:
            raise ValueError(
                f"Variable '{var_name}' must be at most {rules['max_length']} characters"
            )
        
        # Enum validation (allowed values)
        if "allowed_values" in rules and value not in rules["allowed_values"]:
            raise ValueError(
                f"Variable '{var_name}' must be one of {rules['allowed_values']}, got '{value}'"
            )
        
        # Regex pattern validation
        if "pattern" in rules:
            if not re.match(rules["pattern"], str(value)):
                raise ValueError(
                    f"Variable '{var_name}' does not match required pattern: {rules['pattern']}"
                )
    
    def render(self, **kwargs) -> str:
        """
        Render template with validation
        
        Args:
            **kwargs: Variable values
        
        Returns:
            str: Rendered template
        
        Raises:
            ValueError: If validation fails
        """
        # Validate all provided variables
        for var_name, value in kwargs.items():
            self._validate_variable(var_name, value)
        
        # Render using parent class method
        return super().render(**kwargs)
    
    def get_validation_rules(self) -> Dict:
        """Get validation rules for all variables"""
        return self.validators

# Create validated templates
print("VALIDATED TEMPLATES:")
print("=" * 80)

# Credit Risk Template with validation
credit_risk_template = ValidatedPromptTemplate(
    name="credit_risk_validated",
    description="Assess credit risk with validated inputs",
    template="""
Credit Risk Assessment:

Credit Score: {credit_score}
Annual Income: ${annual_income}
Debt-to-Income Ratio: {dti_ratio}%
Risk Category: {risk_category}

Provide risk assessment.
""",
    validators={
        "credit_score": {
            "type": int,
            "min": 300,
            "max": 850
        },
        "annual_income": {
            "type": (int, float),
            "min": 0
        },
        "dti_ratio": {
            "type": (int, float),
            "min": 0,
            "max": 100
        },
        "risk_category": {
            "type": str,
            "allowed_values": ["HIGH_RISK", "MODERATE_RISK", "LOW_RISK"]
        }
    }
)

# Test valid input
print("\nTest 1: Valid input")
try:
    rendered = credit_risk_template.render(
        credit_score=720,
        annual_income=85000,
        dti_ratio=32,
        risk_category="LOW_RISK"
    )
    print("✓ Success")
except ValueError as e:
    print(f"✗ Error: {e}")

# Test invalid credit score
print("\nTest 2: Invalid credit score (too high)")
try:
    rendered = credit_risk_template.render(
        credit_score=1000,  # Invalid: max is 850
        annual_income=85000,
        dti_ratio=32,
        risk_category="LOW_RISK"
    )
    print("✓ Success")
except ValueError as e:
    print(f"✓ Correctly rejected: {e}")

# Test invalid risk category
print("\nTest 3: Invalid risk category")
try:
    rendered = credit_risk_template.render(
        credit_score=720,
        annual_income=85000,
        dti_ratio=32,
        risk_category="MEDIUM"  # Invalid: not in allowed_values
    )
    print("✓ Success")
except ValueError as e:
    print(f"✓ Correctly rejected: {e}")

# Test wrong type
print("\nTest 4: Wrong type for credit_score")
try:
    rendered = credit_risk_template.render(
        credit_score="excellent",  # Invalid: should be int
        annual_income=85000,
        dti_ratio=32,
        risk_category="LOW_RISK"
    )
    print("✓ Success")
except ValueError as e:
    print(f"✓ Correctly rejected: {e}")

print("\n" + "=" * 80)

### Key Takeaways

✓ **Type safety** - Prevent wrong data types  
✓ **Range validation** - Numeric bounds checking  
✓ **Enum constraints** - Only allowed values  
✓ **Pattern matching** - Regex validation for formats  
✓ **Early error detection** - Fail fast before API calls

## LAB SUMMARY

### Template Patterns Mastered

| Challenge | Pattern | Key Benefit | Production Use |
|-----------|---------|-------------|----------------|
| 1 | Basic Templates | Variable substitution, reusability | Foundation for all templates |
| 2 | Validated Templates | Type safety, input validation | Prevent invalid data |

### Production Checklist

Before deploying a template:

- [ ] Template has clear, descriptive name
- [ ] All variables have validators
- [ ] Input/output examples provided
- [ ] Tested with edge cases
- [ ] Documented (description, usage, performance)
- [ ] Versioned (semantic versioning)
- [ ] Changelog updated
- [ ] Registered in library
- [ ] Exported to version control (Git)
- [ ] Team reviewed and approved

